In [1]:
%cd ..

/fs01/home/abbasgln/codes/medAI/projects/tta


In [2]:
import os
from dotenv import load_dotenv
# Loading environment variables
load_dotenv()

import torch
import torch.nn as nn
import typing as tp
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
from dataclasses import dataclass, field
import logging
import wandb

import medAI
from medAI.utils.setup import BasicExperiment, BasicExperimentConfig

from utils.metrics import MetricCalculator

from timm.optim.optim_factory import create_optimizer

from einops import rearrange, repeat
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import timm

from copy import copy, deepcopy
import pandas as pd

from datasets.datasets import ExactNCT2013RFImagePatches, ExactNCT2013RFPatchesWithSupportPatches, SupportPatchConfig
from medAI.datasets.nct2013 import (
    KFoldCohortSelectionOptions,
    LeaveOneCenterOutCohortSelectionOptions, 
    PatchOptions
)


In [3]:
LEAVE_OUT='CRCEO'

## Data MEMO

In [4]:
num_support_patches = 2


from ttt_experiment import TTTExperimentConfig
config = TTTExperimentConfig(
    cohort_selection_config=LeaveOneCenterOutCohortSelectionOptions(leave_out=f"{LEAVE_OUT}"),
    num_support_patches=num_support_patches,
    include_query_patch=False,
    )

from baseline_experiment import BaselineConfig
from torchvision.transforms import v2 as T
from torchvision.tv_tensors import Image as TVImage

class Transform:
    def __init__(selfT, augment=False):
        selfT.augment = augment
        selfT.size = (256, 256)
    
    def __call__(selfT, item):
        patch = item.pop("patch")
        patch = copy(patch)
        patch = (patch - patch.min()) / (patch.max() - patch.min()) \
            if config.instance_norm else patch
        patch = TVImage(patch)
        patch = T.Resize(selfT.size, antialias=True)(patch).float()

        # Support patches
        support_patches = item.pop("support_patches")
        support_patches = copy(support_patches)
        # Normalize support patches along last two dimensions
        support_patches = (support_patches - support_patches.min(axis=(1, 2), keepdims=True)) \
        / (support_patches.max(axis=(1,2), keepdims=True) \
            - support_patches.min(axis=(1, 2), keepdims=True)) if config.instance_norm else support_patches
        support_patches = TVImage(support_patches)
        support_patches = T.Resize(selfT.size, antialias=True)(support_patches).float()
        
        # Augment support patches
        transform = T.Compose([
            T.RandomAffine(degrees=0, translate=(0.2, 0.2)),
            T.RandomErasing(p=0.5, scale=(0.02, 0.1), ratio=(0.3, 3.3), value=0.5),
            T.RandomHorizontalFlip(p=0.5),
            T.RandomVerticalFlip(p=0.5),
        ])   
        support_patches_aug1, support_patches_aug2 = transform(support_patches, support_patches)
        
        if selfT.augment:
            patch = transform(patch)
        
        label = torch.tensor(item["grade"] != "Benign").long()
        return support_patches_aug1, support_patches_aug2, patch, label, item


# val_ds = ExactNCT2013RFPatchesWithSupportPatches(
#     split="val",
#     transform=Transform(),
#     cohort_selection_options=config.cohort_selection_config,
#     patch_options=config.patch_config,
#     support_patch_config=SupportPatchConfig(
#         num_support_patches=config.num_support_patches,
#         include_query_patch=config.include_query_patch
#     ),
#     debug=config.debug,
# )
        
test_ds = ExactNCT2013RFPatchesWithSupportPatches(
    split="test",
    transform=Transform(),
    cohort_selection_options=config.cohort_selection_config,
    patch_options=config.patch_config,
    support_patch_config=SupportPatchConfig(
        num_support_patches=config.num_support_patches,
        include_query_patch=config.include_query_patch
    ),
    debug=config.debug,
)


# val_loader = DataLoader(
#     val_ds, batch_size=config.batch_size_test, shuffle=config.shffl_test, num_workers=4
# )

test_loader = DataLoader(
    test_ds, batch_size=config.batch_size_test, shuffle=config.shffl_test, num_workers=4
)


Computing positions test: 100%|██████████| 1469/1469 [02:04<00:00, 11.84it/s]


## Model

In [5]:
from models.ttt_model import TTTModel, TTTConfig
adaptation_steps=1
adaptation_lr=0.001
beta_byol=1.0

model_config = TTTConfig(adaptation_steps=adaptation_steps, adaptation_lr=adaptation_lr, beta_byol=beta_byol)

# Create the model
model: nn.Module = TTTModel(model_config)

CHECKPOINT_PATH = os.path.join(os.getcwd(), f'logs/tta/ttt_2+0sprt_JT_0.1beta_loco/ttt_2+0sprt_JT_0.1beta_loco_{LEAVE_OUT}/', 'best_model.ckpt')

model.load_state_dict(torch.load(CHECKPOINT_PATH)['model'])
model.eval()
model.cuda()

a = True

## Run test MEMO

In [6]:
loader = test_loader


metric_calculator = MetricCalculator()
desc = "test"

model.eval()
criterion = nn.CrossEntropyLoss()

for i, batch in enumerate(tqdm(loader, desc=desc)):
    images_aug_1, images_aug_2, images, labels, meta_data = batch
    images_aug_1 = images_aug_1.cuda()
    images_aug_2 = images_aug_2.cuda()
    images = images.cuda()
    labels = labels.cuda()
    
    (logits, 
    total_loss_avg,
    ce_loss_avg,
    byol_loss_avg
    ) = model(images_aug_1, images_aug_2, images, labels, training=False)
        
    # Update metrics   
    metric_calculator.update(
        batch_meta_data = meta_data,
        probs = nn.functional.softmax(logits, dim=-1).detach().cpu(),
        labels = labels.detach().cpu(),
    )

test:   0%|          | 0/1715 [00:00<?, ?it/s]

In [7]:
# Log metrics every epoch
metrics = metric_calculator.get_metrics()

# Update best score
(
    best_score_updated,
    best_score
    ) = metric_calculator.update_best_score(metrics, desc)

best_score_updated = copy(best_score_updated)
best_score = copy(best_score)
        
# Log metrics
metrics_dict = {
    f"{desc}/{key}": value for key, value in metrics.items()
    }
metrics_dict.update(best_score) if desc == "val" else None 


# wandb.log(
#     metrics_dict,
#     )
metrics_dict

{'test/patch_auroc': tensor(0.7108),
 'test/patch_accuracy': tensor(0.6311),
 'test/all_inv_patch_auroc': tensor(0.6423),
 'test/all_inv_patch_accuracy': tensor(0.6206),
 'test/core_auroc': tensor(0.8053),
 'test/core_accuracy': tensor(0.7023),
 'test/all_inv_core_auroc': tensor(0.7208),
 'test/all_inv_core_accuracy': tensor(0.6888)}

In [8]:
import wandb
group=f"offline_ttt_{num_support_patches}+0sprt_0.1beta_e-3adptlr_loco"
name= group + f"_{LEAVE_OUT}"
wandb.init(project="tta", entity="mahdigilany", name=name, group=group)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: mahdigilany. Use `wandb login --relogin` to force relogin


In [9]:
metrics_dict.update({"epoch": 0})
wandb.log(
    metrics_dict,
    )
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁
test/all_inv_core_accuracy,▁
test/all_inv_core_auroc,▁
test/all_inv_patch_accuracy,▁
test/all_inv_patch_auroc,▁
test/core_accuracy,▁
test/core_auroc,▁
test/patch_accuracy,▁
test/patch_auroc,▁
epoch,0
test/all_inv_core_accuracy,0.68884
